<a href="https://colab.research.google.com/github/bdugick89/Data-Science-Bootcamp/blob/main/SQL_1_Chinook_project_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project SQL - Chinook


## Chinook data set

See the lecture on SQLite3 using the Chinook data set to set up the software, database, and tables, as well as for the links to ancillary information about the data set.


## Come up with questions about your data
Have a look at the Entity-Relation ( ER ) diagram to help come up with questions.

* What sort of information is in this dataset?
* How many records are there?
* How many different countries (states, counties, cities, etc) have records in this data set?


If you are stuck, here are some ideas for questions:
- https://github.com/LucasMcL/15-sql_queries_02-chinook/blob/master/README.md
- [Using the R language]( https://rpubs.com/enext777/636199 )
- [Search Google]( https://www.google.com/search?q=chinook+database+questions )



In [ ]:
%%capture
%%bash
apt-get update
apt-get install -y sqlite3



In [ ]:
%%bash

[ -f chinook.zip ] ||
  curl -s -O https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
unzip -l chinook.zip



Archive:  chinook.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
   884736  2015-11-29 10:53   chinook.db
---------                     -------
   884736                     1 file


In [ ]:
!unzip -u chinook.zip


Archive:  chinook.zip
  inflating: chinook.db              


In [ ]:
/*Sales By State */

WITH USA_STATE AS (
    SELECT
        SUM(i.total) AS total_sales,
        i.BillingState AS State,
		'USA' AS Origin
    FROM invoices i
    WHERE i.BillingCountry = "USA"
    GROUP BY i.BillingState
),
GLOBAL AS (
    SELECT
        SUM(i.total) AS total_sales,
        i.BillingCountry AS country,
		'Global' AS Origin
    FROM invoices i
    WHERE i.BillingCountry <> "USA"
    GROUP BY i.BillingCountry
)
SELECT * FROM USA_STATE
UNION
SELECT * FROM GLOBAL
ORDER BY Origin DESC;


/*Invoices with Threasholds */

WITH InvoiceTotal AS (
    SELECT
        COUNT(ii.quantity) AS purchase_count,
        i.InvoiceId,
        i.total as purchase_total,
        i.CustomerId
    FROM invoice_items ii
    JOIN invoices i ON ii.InvoiceId = i.InvoiceId
    WHERE i.total BETWEEN 15 AND 25
    GROUP BY i.InvoiceId, i.CustomerId
)
, TotalStats AS (
    SELECT
        MAX(purchase_total) AS Max_Sale_metric,
        MIN(purchase_total) AS Min_Sale_metric
    FROM InvoiceTotal
)
SELECT
    it.InvoiceId,
    it.purchase_total,
    c.city,
    c.state,
	 AVG(it.purchase_total) OVER() AS Average_Sale_Per_Invoice_metric,
	 ts.Max_Sale_metric ,
    ts.Min_Sale_metric
FROM InvoiceTotal it
CROSS JOIN TotalStats ts
LEFT JOIN customers c ON c.CustomerId = it.CustomerId
WHERE c.state IS NOT NULL
ORDER BY it.purchase_total ASC;

/*Artst and Album List */
SELECT
    CASE
        WHEN ROW_NUMBER() OVER (PARTITION BY al.AlbumId ORDER BY ar.name, t.TrackId) = 1 THEN al.Title
        ELSE ''
    END AS Album_Title,
    CASE
        WHEN ROW_NUMBER() OVER (PARTITION BY al.AlbumId ORDER BY ar.name, t.TrackId) = 1 THEN ar.name
        ELSE ''
    END AS Artist_Name,
    t.name AS Track_Name,
    g.name AS Genre,
    t.TrackId
FROM ARTISTS ar
LEFT JOIN ALBUMS al
    ON ar.ArtistId = al.ArtistId
LEFT JOIN TRACKS t
    ON al.AlbumId = t.AlbumId
LEFT JOIN Genres g
    ON t.GenreId = g.GenreId
ORDER BY ar.name, al.Title, t.TrackId;

/*Current and Future Sales*/

	WITH SalesData AS (
    SELECT
        SUM(ii.quantity) AS invoice_items_quantity,
        SUM(UnitPrice) AS invoice_items_unit_price,
        strftime('%Y', i.InvoiceDate) AS year,
        (strftime('%m', i.InvoiceDate) + 2) / 3 AS quarter
    FROM
        invoice_items ii
    JOIN
        invoices i ON ii.InvoiceId = i.InvoiceId
    GROUP BY
        strftime('%Y', i.InvoiceDate),
        (strftime('%m', i.InvoiceDate) + 2) / 3
),
NextYearQuarters AS (
    SELECT
        strftime('%Y', date('now', '+1 year')) AS year,
        1 AS quarter
    UNION ALL
    SELECT
        strftime('%Y', date('now', '+1 year')) AS year,
        2 AS quarter
    UNION ALL
    SELECT
        strftime('%Y', date('now', '+1 year')) AS year,
        3 AS quarter
    UNION ALL
    SELECT
        strftime('%Y', date('now', '+1 year')) AS year,
        4 AS quarter
)
SELECT
    invoice_items_quantity,
    invoice_items_unit_price,
    year,
    quarter
FROM
    SalesData
UNION ALL
SELECT
    0 AS invoice_items_quantity,
    0 AS invoice_items_unit_price,
    year,
    quarter
FROM
    NextYearQuarters

ORDER BY
    year DESC,
    quarter DESC;
